<a href="https://colab.research.google.com/github/ArunavK/stat-analysis/blob/master/the_monopoly_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Program to find the most likely position in a Monopoly game

Creator: **Arunabh Kashyap**

---

A simple python script to find out the most likely position where a player lands on in the classic Monopoly game, marketed by Hasbro.

The following points were noted about the game:

1. The game rules are very complex and the outcome of the game heavily depends on the players own choices and preferences. Moreover, the game provides a system of trading cards with each other which is impossible to model mathematically. As such, it is well-nigh impossible to predict victory _even if the dice outcomes are known well in advance_.
2. The movement of the player's piece is _largely independent_ of the player's choices. It is uniquely determined by the outcome of the dice, which is very easily modelled as the sum of 2 uniformly distributed random variables between 1 and 6. As such, a simulation of the game with the dice as the random variables has been created and the positions of the player's piece can be tracked to find the positions where it landed most frequently. Extending this result to multiple players will not change the result as the player's pieces move independently of each other.  
The knowledge of the most likely position will greatly aid (human) players as one could then buy more property in that area and thus, increase his chances of victory.
4. One aspect of the player's movement, however, **is** dependent on the player's preferences, which is the jail mechanic. The player has the option either to stay in the jail (upto 3 turns) or bail out (either with money or chance cards). In this model, the player chooses to (and has the means to) get out of jail every time.  
The result of this assumption is that the frequency of the player being in the jail block (block 10) is reduced greatly. I feel that such an effect is by no means detrimental to the value of the result as the jail block being most likely is not particularly useful information anyway (since the jail cannot be bought).
5. Another aspect that affects the movement of the piece is the Chance and Community Chest cards. A large majority of the cards affect the money in the players hands which does not affect our model anyway. However, some of the cards will force the piece to move to a certain location or go to jail. Such movements are modelled using another random number with the probabilites and positions based on the official game.
6. A single Monopoly game with 4 players is assumed to go on for 3 hours. There are about 5 throws of the dice in a minute. So, the number of throws by a single player is $n = \frac{3*60*5}{4} = 225$. The simulation is run for 225 turns in each game.


## Importing Libraries

In [80]:
import numpy as np
import random as rnd
import os

## Setting up the game

### Modelling the board

The board is modelled as a NumPy array of size $40 \times 1$.  
"Go" block begins at 0 and goes up to "Boardwalk" at 39

### Modelling the dice roll

The 2 dice are modelled as a random integer between 1 and 6. Only their sum is passed on as a return type as the individual values are unimportant

In [81]:
def dice():
  d1 = rnd.randint(1, 6)
  d2 = rnd.randint(1, 6)
  roll = d1 + d2
  return roll

### Modelling the cards

The _Chance_ and _Community Chest_ cards are modelled as functions `chance(pos)` and `community_chest()` respectively which change the positions based on uniformly distributed random variable. The `chance` function takes the current position as an argument in order to implement the "Go back 3 spaces" mechanic. The `community_chest` function doesn't require any such arguments.  
There are 16 _Chance_ and _Community Chest_ cards each and all of them are assumed to have an equal probability of being picked up. Hence, the probablity of each is divided into an interval of size $\frac{1}{16}$.  
Only the cards which cause the position of the piece to change are modelled as the remaining cards do not concern our problem statement.


In [82]:
def chance(pos):
  x = rnd.random()
  if (x < 1/16):
    # Go back 3 spaces
    pos = pos - 3
  elif (1/16 <= x < 2/16):
    # Take a trip to Pennsylvania Railroad
    pos = 15
  elif (2/16 <= x < 3/16):
    # Go to Jail
    pos = 10
  elif (3/16 <= x < 4/16):
    # Advance to Illinois Avenue
    pos = 24
  elif (4/16 <= x < 5/16):
    # Advance to St. Charles Place
    pos = 11
  elif (5/16 <= x < 6/16):
    # Advance to Go
    pos = 0
  elif (6/16 <= x < 7/16):
    # Advance to Boardwalk
    pos = 39
  return pos

In [83]:
def community_chest(pos):
  x = rnd.random()
  if (x < 1/16):
    # Advance to "Go"
    pos = 0
  elif (1/16 <= x < 2/16):
    # Go to Jail
    pos = 10
  elif (2/16 <= x < 3/16):
    # Go back to Mediterranean Avenue
    pos = 1

  return pos

### Modelling the movement of the piece

Under normal circumstances, the position of the piece is just the sum of the current position and the roll of the dice.  
The `positioning` function is used to determine the correct position under the following conditions:

1. If the piece makes a full circle around the board, the position is determined by substracting from 40.
2. If the piece falls on position 30 (Go to jail), it is moved to the Jail block.
3. If the piece falls on one of the three _Chance_ blocks, `chance` function is called to find the correct position.
4. If the piece falls on a _Community Chest_ block, the `community_chest` function is called to find the correct position.


In [84]:
def positioning(pos):
  if pos > 39:
    pos = pos - 40
  
  if (pos == 30):
    pos = 10
  
  elif (pos in (7, 22, 36)):
    pos = chance(pos)

  elif (pos in (2, 17, 33)):
    pos = community_chest(pos)

  return pos


## Iterating through the game

- The game is modelled as one single player moving through the board for `game` number of throws.
- Everytime the player lands on a block, the value of that array element is increased by 1.
- At the beginning of each game, the position is reset to "Go"
- `iterations` number of games are played.
- The status of the maximum block after every 10% progress is shown

In [85]:
def simulate(iterations, game, max_pos, max_freq):
  for i in range(iterations):
    pos = 0
    for j in range(game):
      roll = dice()
      pos = pos + roll
      pos = positioning(pos)
      
      A[pos] = A[pos] + 1
    
    if ((i+1) % (iterations/5) == 0):
      print("iteration {}: {}% completed".format(i+1, (i+1)*100/iterations))
  


## Running the simulations

In [86]:
iterations = 1000
game = 225
samples = 5
A = np.zeros((40, 1))
max_pos = np.zeros((5, 5))
max_freq = np.zeros((5, 5))

rnd.seed(os.urandom(2))

for i in range(samples):
  A = np.zeros((40, 1))
  print("Sample {}".format(i+1))
  simulate(iterations, game, max_pos, max_freq)

  max_pos[i, 0:5] = np.reshape(np.flipud(np.argsort(A, axis=0))[0:5], (5,))
  max_freq[i, 0:5] = np.reshape(np.flipud(np.sort(A, axis=0))[0:5], (5,))

Sample 1
iteration 200: 20.0% completed
iteration 400: 40.0% completed
iteration 600: 60.0% completed
iteration 800: 80.0% completed
iteration 1000: 100.0% completed
Sample 2
iteration 200: 20.0% completed
iteration 400: 40.0% completed
iteration 600: 60.0% completed
iteration 800: 80.0% completed
iteration 1000: 100.0% completed
Sample 3
iteration 200: 20.0% completed
iteration 400: 40.0% completed
iteration 600: 60.0% completed
iteration 800: 80.0% completed
iteration 1000: 100.0% completed
Sample 4
iteration 200: 20.0% completed
iteration 400: 40.0% completed
iteration 600: 60.0% completed
iteration 800: 80.0% completed
iteration 1000: 100.0% completed
Sample 5
iteration 200: 20.0% completed
iteration 400: 40.0% completed
iteration 600: 60.0% completed
iteration 800: 80.0% completed
iteration 1000: 100.0% completed


## Extracting the results

In [87]:
print(max_pos)
print(max_freq)

[[10. 15. 24.  0. 19.]
 [10. 24. 15.  0. 19.]
 [10. 24. 15. 19.  0.]
 [10. 15.  0. 24. 19.]
 [10. 15. 24. 19.  0.]]
[[13257.  7089.  7037.  6922.  6835.]
 [13345.  7149.  7139.  7011.  6811.]
 [13531.  7122.  7016.  6887.  6773.]
 [13128.  7179.  6968.  6968.  6838.]
 [13328.  7065.  7026.  6854.  6837.]]


## Future upgrades

- The jail mechanic will be modelled on position 10, linked to the go-to-jail mechanic on position 30.
- Community Chest and Chance cards will be modelled as probability of:
  1. Going to jail
  2. Going to position 0
  3. Going back 3 spaces
- CUDA enabled hardware acceleration may be implemented to improve performance
- Some thought needs to be put into the legitimacy of the operation. Maybe some blocks are more frequent towards the end of the game?
- Multiple players can be modelled in version 2.0 and the chance and community cards will be modelled as initially shuffled lists. Variations in the result may be seen as the lists are ordered and cycled through instead of random picking.